This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix) NAME FOR THE FILE, not the title
GRAPH_NAME = "gdp_contrib"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [7]:
today = dt.date.today()

contrib_df = pd.read_csv(
    ut.get_repo_root() / "code" / "data" / "gdp_contrib.csv")

contrib_df = contrib_df.dropna(axis="columns").rename(columns=
    {"Unnamed: 0": "Year",
    "Unnamed: 1": "Quarter"},
)

contrib_df["Date"] = pd.to_datetime(contrib_df["Year"].astype(str) + contrib_df["Quarter"])

contrib_df = contrib_df.set_index("Date").drop(columns=["Year", "Quarter"])
contrib_df

C:\Users\Bobby\AppData\Local\Temp\ipykernel_14192\3013989618.py:11: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Real GDP,Consumption,Nonresidential Investment,Residential Investment,Change in Inventories,Net Exports,Government Spending
Date,,,,,,,
2023-01-01,2.9,2.99,1.13,-0.28,-2.18,0.58,0.69
2023-04-01,2.5,1.02,1.47,0.16,-0.36,-0.30,0.56
2023-07-01,4.7,2.08,0.25,0.44,0.97,0.09,0.87
2023-10-01,3.4,2.04,0.76,0.06,-0.17,-0.06,0.79
2024-01-01,0.8,1.17,0.22,0.32,-0.83,-0.42,0.39
2024-04-01,3.6,2.61,0.35,-0.08,1.17,-1.04,0.57
2024-07-01,3.3,2.66,0.48,-0.20,-0.11,-0.41,0.92
2024-10-01,1.9,2.61,-0.51,0.17,-0.91,-0.06,0.57
2025-01-01,-0.6,0.42,1.24,-0.04,2.58,-4.68,-0.17


Index(['Real GDP', 'Consumption', 'Nonresidential  Investment',
       'Residential Investment', 'Change in Inventories', 'Net Exports',
       'Government Spending'],
      dtype='object')

In [39]:
fig = go.Figure()


# All colors with "names"
colors_dict = {
    'orange':  '#CF8B40',
    'blue': '#2A547E' ,
    'dark_gray': '#879396',
    'beige':'#F7CAC9',
    'light_blue': '#7095cf',
    'light_gray': '#D3D3D3',
    'dark_blue_text': '#163855'
}

# Ordered colors for order of trace coloring
custom_colorway = [
    colors_dict['orange'],
    colors_dict['dark_blue_text'],
    colors_dict['light_blue'],
    colors_dict['blue'],
    colors_dict['light_gray'],
    colors_dict['dark_gray']
]


# Add stacked bars for all columns except 'Real GDP', using the custom color sequence
for i, col in enumerate(bar_columns):
    fig.add_trace(
        go.Bar(
            x=contrib_df.index,
            y=contrib_df[col],
            name=col,
            marker_color=custom_colorway[i % len(custom_colorway)]
        )
    )

# Add Real GDP as a line on top
fig.add_trace(
    go.Scatter(
        x=contrib_df.index,
        y=contrib_df["Real GDP"],
        name="Real GDP",
        mode="lines+markers",
        line=dict(color="black", width=3)
    )
)

# Add annotation for GDPNow at the final point on the x axis
fig.add_annotation(
    x=contrib_df.index[-1],
    y=4.5,
    text="GDPNow Q3<br>3.9%",
    showarrow=False,
    font=dict(color="black", size=11),
    xanchor="center",
    yanchor="bottom"
)

fig.update_layout(
    legend=dict(
        orientation="v",
        x=1.02,
        y=0.5,
        xanchor="left",
        yanchor="middle"
    ),
    barmode='relative',
    title="Real GDP by Component <br><sup>Quarterly, Annualized, Seasonally Adjusted</sup>",
    xaxis_title="Date",
    yaxis_title="Contribution to Percent Change",
    legend_title="Component"
)

fig.update_xaxes(
    dtick="M12",
    tickformat="%Y",
)
fig.update_yaxes(ticksuffix="%", tickformat=".f")


fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")
fig.show()